In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install tensorflow-text
!pip install sentence-transformers

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/BTP/datainsta_shuff.csv")
df

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    sta = word_tokenize(sentence)
    st = re.compile(r'[-.,:;!?()|0-9]')
    tokens_without_sw = [word for word in sta if word not in stopwords.words('english')]
    post_punc = ""
    for words in tokens_without_sw:
      word=st.sub("", words)
      if len(word)>0:
        post_punc = post_punc + word + ' '
        # post_punc = post_punc + ' '
    return post_punc
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
import numpy as np
xx=list(df['TEXT'])
X=[]
for sen in xx:
    X.append(preprocess_text(sen))
tags=list(df['hashtags'].astype(str))
tag_freq={}
templ=[]
num1=np.array(df['number_of_tags'])
Y=[]
print(len(tags))
p=0
for i in range(len(tags)):
  k=0
  temp=0
  for j in range(len(tags[i])):
    if tags[i][j]=='#':
      k=1
      temp=j+1
    elif tags[i][j]==',' or tags[i][j]==']':
      if temp<=j-1:
        # print(tags[i][temp:j-1])
        if (tags[i][temp:j-1] in tag_freq):
          k=tag_freq.get(tags[i][temp:j-1])
          templ.append(k)
        else:
          tag_freq[tags[i][temp:j-1]]=p
          templ.append(p)
          p=p+1
        k=0
        
      else:
        num1[i]-=1
  
  
  Y.append(templ)
  templ=[]
  

3000


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0,shuffle= False)

#print(Y)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Apply SBERT on the sectioned data and download the files



In [ ]:
import tensorflow as tf
import tensorflow_text as text

In [ ]:
training_set_embedding = model.encode(X_train , convert_to_tensor = True)

In [ ]:
test_set_embedding = model.encode(X_test, convert_to_tensor = True)

In [ ]:
from google.colab import files
training_set_embedding=training_set_embedding.numpy()
test_set_embedding = test_set_embedding.numpy()
dd = pd.DataFrame(test_set_embedding)
dd1 = pd.DataFrame(training_set_embedding)
dd1.to_csv('embedding_train.csv', encoding = 'utf-8-sig') 
dd.to_csv('embedding_test.csv', encoding = 'utf-8-sig') 
files.download('embedding_train.csv')
files.download('embedding_test.csv')